In [31]:
import os

import numpy as np
from PIL import Image
from tqdm import tqdm 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage import img_as_float
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras import backend as K
from keras.models import Model
from keras.utils import conv_utils
from keras.engine.topology import Layer
from keras.datasets import mnist, cifar10
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, PReLU, BatchNormalization, Add, LeakyReLU, Dense, Flatten

In [2]:
path_data = '/home/ubuntu/COMP6208-AutoEncoder/several27/data/'
path_open_images_560_420 = path_data + 'open_images_560_420/'
path_open_images_560_420_train = path_data + 'open_images_560_420_train/'
path_open_images_560_420_val = path_data + 'open_images_560_420_val/'

# Data preprocessing

## Load CIFAR

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train.shape, y_train.shape

In [ ]:
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(x_train[i])
    
plt.show()

## Prepare LR CIFAR

In [ ]:
x_train.shape

In [ ]:
def images_resize(images, ratio):
    lr_width = images.shape[1] // ratio
    lr_height = images.shape[2] // ratio
    resized = np.zeros((images.shape[0], lr_width, lr_height, images.shape[3]))
    for i in tqdm(range(images.shape[0])):
        resized[i] = resize(images[i], (lr_width, lr_height))
        
    return resized

In [ ]:
x_train_lr = images_resize(x_train, 4)
x_test_lr = images_resize(x_test, 4)

In [ ]:
x_train_lr.shape

In [ ]:
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(x_train_lr[i])
    
plt.show()

## open_images_560_420 generator (or rather 420*560 ugh)

In [ ]:
def generator_images(path, size=(420, 560, 3), ratio=2, batch_size=32):
    lr_height, lr_width = size[0] // ratio, size[1] // ratio
    
    batch_i = 0
    batch = np.zeros((batch_size, size[0], size[1], size[2]))
    batch_scaled = np.zeros((batch_size, lr_height, lr_width, size[2]))
    
    while True:
        for file in os.listdir(path):
            if not file.endswith('.jpg'):
                continue 
            
            if batch_i == batch_size:
                yield batch_scaled, batch
                
                batch_i = 0
                batch = np.zeros((batch_size, size[0], size[1], size[2]))
                batch_scaled = np.zeros((batch_size, lr_height, lr_width, size[2]))
            
            file_path = path + file
            img = img_as_float(Image.open(file_path))
            if len(img.shape) == 2: 
                 img = np.asarray(np.dstack((img, img, img))) 
            
            batch[batch_i] = img
            batch_scaled[batch_i] = resize(img, (lr_height, lr_width))
            
            batch_i += 1

In [ ]:
def count_images(path):
    return sum([1 for file in os.listdir(path) if file.endswith('.jpg')])

In [ ]:
count_images(path_open_images_560_420_train), count_images(path_open_images_560_420_val)

# Network description 

![](srgan_architecture.png)

# Network implementation 

## Generator network

1. Input - Image LR
2. Conv, kernel 3x3, 64 feature maps, what's s1?
3. ParametricReLU
4. B=16 residual blocks
    1. Conv with 3x3 kernel, 64 feature maps and stride 1
    2. Batch Normalization (https://keras.io/layers/normalization/)
    3. ParametricReLU
    4. Conv with 3x3 kernel, 64 feature maps and stride 1
    5. Batch Normalization
    6. Elementwise sum (https://keras.io/layers/merge/ add)
5. Conv with 3x3 kernel, 64 feature maps and stride 1
6. Batch Normalization
7. Elementwise sum (https://keras.io/layers/merge/ add)
8. Shuffle block x2 (? how it's called ?)
    1. Conv with 3x3 kernel, 256 feature maps and stride 1
    2. PixelShuffler x2 https://gist.github.com/t-ae/6e1016cc188104d123676ccef3264981
    3. ParametricReLU
9. Conv with 9x9 kernel, 3 feature maps and stride 1
10. Estimated SR (super-resolved) image

Is the padding `same` a correct one? Otherwise the dims are not the same
Is the number of pixel shufflers the scaling ratio? 

In [15]:
# from https://gist.github.com/t-ae/6e1016cc188104d123676ccef3264981

class PixelShuffler(Layer):
    def __init__(self, size=(2, 2), data_format=None, **kwargs):
        super(PixelShuffler, self).__init__(**kwargs)
        self.data_format = conv_utils.normalize_data_format(data_format)
        self.size = conv_utils.normalize_tuple(size, 2, 'size')

    def call(self, inputs):

        input_shape = K.int_shape(inputs)
        if len(input_shape) != 4:
            raise ValueError('Inputs should have rank ' +
                             str(4) +
                             '; Received input shape:', str(input_shape))

        if self.data_format == 'channels_first':
            batch_size, c, h, w = input_shape
            if batch_size is None:
                batch_size = -1
            rh, rw = self.size
            oh, ow = h * rh, w * rw
            oc = c // (rh * rw)

            out = K.reshape(inputs, (batch_size, rh, rw, oc, h, w))
            out = K.permute_dimensions(out, (0, 3, 4, 1, 5, 2))
            out = K.reshape(out, (batch_size, oc, oh, ow))
            return out

        elif self.data_format == 'channels_last':
            batch_size, h, w, c = input_shape
            if batch_size is None:
                batch_size = -1
            rh, rw = self.size
            oh, ow = h * rh, w * rw
            oc = c // (rh * rw)

            out = K.reshape(inputs, (batch_size, h, w, rh, rw, oc))
            out = K.permute_dimensions(out, (0, 1, 3, 2, 4, 5))
            out = K.reshape(out, (batch_size, oh, ow, oc))
            return out

    def compute_output_shape(self, input_shape):

        if len(input_shape) != 4:
            raise ValueError('Inputs should have rank ' +
                             str(4) +
                             '; Received input shape:', str(input_shape))

        if self.data_format == 'channels_first':
            height = input_shape[2] * self.size[0] if input_shape[2] is not None else None
            width = input_shape[3] * self.size[1] if input_shape[3] is not None else None
            channels = input_shape[1] // self.size[0] // self.size[1]

            if channels * self.size[0] * self.size[1] != input_shape[1]:
                raise ValueError('channels of input and size are incompatible')

            return (input_shape[0],
                    channels,
                    height,
                    width)

        elif self.data_format == 'channels_last':
            height = input_shape[1] * self.size[0] if input_shape[1] is not None else None
            width = input_shape[2] * self.size[1] if input_shape[2] is not None else None
            channels = input_shape[3] // self.size[0] // self.size[1]

            if channels * self.size[0] * self.size[1] != input_shape[3]:
                raise ValueError('channels of input and size are incompatible')

            return (input_shape[0],
                    height,
                    width,
                    channels)

    def get_config(self):
        config = {'size': self.size,
                  'data_format': self.data_format}
        base_config = super(PixelShuffler, self).get_config()

        return dict(list(base_config.items()) + list(config.items()))

In [38]:
def srgan_generator(input_shape, input_=None):
    kernel_size = (3, 3)
    kernel_size_last = (9, 9)
    features = 64
    features_shuffle = 256
    features_last = 3
    B = 16

    # 1, 2, 3
    input_1 = input_ if input_ is not None else Input(shape=input_shape)
    conv2d_2 = Conv2D(filters=features, kernel_size=kernel_size, strides=(1, 1), padding='same')(input_1)
    prelu_3 = PReLU()(conv2d_2)

    # 4 - residual blocks
    last_layer = prelu_3
    for _ in range(B):
        conv2d_4_A = Conv2D(filters=features, kernel_size=kernel_size, strides=(1, 1), padding='same')(last_layer)
        bn_4_B = BatchNormalization()(conv2d_4_A)
        prelu_4_C = PReLU()(bn_4_B)
        conv2d_4_D = Conv2D(filters=features, kernel_size=kernel_size, strides=(1, 1), padding='same')(prelu_4_C)
        bn_4_E = BatchNormalization()(conv2d_4_D)
        add_4_F = Add()([last_layer, bn_4_E])

        last_layer = add_4_F

    # 5, 6, 7
    conv2d_5 = Conv2D(filters=features, kernel_size=kernel_size, strides=(1, 1), padding='same')(last_layer)
    bn_6 = BatchNormalization()(conv2d_5)
    add_7 = Add()([prelu_3, bn_6])

    # 8 - shuffle block
    last_layer = add_7
    for _ in range(1):
        conv2d_8_A = Conv2D(filters=features_shuffle, kernel_size=kernel_size, strides=(1, 1), padding='same')(last_layer)
        shuffler_8_B = PixelShuffler()(conv2d_8_A)
        prelu_8_C = PReLU()(shuffler_8_B)

        last_layer = prelu_8_C

    # 9 
    conv2d_5 = Conv2D(filters=features_last, kernel_size=kernel_size_last, strides=(1, 1), 
                      padding='same')(last_layer)

    model_generator = Model(input_1, conv2d_5)
    model_generator.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    
    return model_generator

In [17]:
# model_generator = srgan_generator((x_train_lr.shape[1], x_train_lr.shape[2], x_train_lr.shape[3]))
model_generator = srgan_generator((210, 280, 3))

In [ ]:
model_generator.summary()

In [ ]:
train_version = 4

In [ ]:
checkpointer = ModelCheckpoint(filepath='data/srgan_generator_weights_%s.{epoch:03d}_{val_acc:.4f}.hdf5' % train_version, 
                               verbose=1, save_best_only=False)
tb_callback = TensorBoard(log_dir='data/tensorboard/', histogram_freq=0, write_graph=True, write_images=True)

In [ ]:
# model_generator.fit(x_train_lr, x_train, epochs=100, validation_data=(x_test_lr, x_test), callbacks=[tb_callback])

batch_size = 1
n_train = 30 # count_images(path_open_images_560_420_train)
n_val = 3 # count_images(path_open_images_560_420_val)
with tf.device('/gpu:0'):
    model_generator.fit_generator(generator_images(path_open_images_560_420_train, batch_size=batch_size), 
                                  steps_per_epoch=n_train // batch_size,
                                  validation_data=generator_images(path_open_images_560_420_val, 
                                                                   batch_size=batch_size),
                                  validation_steps=n_val // batch_size, epochs=1, callbacks=[tb_callback])

In [ ]:
model_generator.save('data/srgan_generator_%s.model' % train_version)

In [ ]:
model_generator.load_weights('data/srgan_generator_%s.model' % train_version)

In [ ]:
for g, type_ in [(generator_images(path_open_images_560_420_train, batch_size=1), 'train'), 
                 (generator_images(path_open_images_560_420_val, batch_size=batch_size), 'test')]:
    plt.figure(type_, figsize=(15, 30))
    for i, (x_lr, x) in enumerate(g):
        x_lr_ = (x_lr[0] * 255).astype(np.uint8)
        x_ = (x[0] * 255).astype(np.uint8)

        plt.subplot(10, 3, (i * 3) + 1)
        plt.imshow(x_lr_)

        plt.subplot(10, 3, (i * 3) + 2)
        plt.imshow(x_)

        plt.subplot(10, 3, (i * 3) + 3)
        plt.imshow((model_generator.predict(x_lr)[0] * 255).astype(np.uint8))

        if i >= 9:
            break
            
    plt.savefig('data/srgan_generator_%s_%s.png' % (train_version, type_))
    plt.show()

In [ ]:
for x, x_lr type_ in [(x_train, x_train_lr, 'train'), (x_test, x_test_lr, 'test')]:
    plt.figure(figsize=(15, 30))
    for i in range(10):
        plt.subplot(10, 3, (i * 3) + 1)
        plt.imshow(x_lr[i])

        plt.subplot(10, 3, (i * 3) + 2)
        plt.imshow(x[i])

        plt.subplot(10, 3, (i * 3) + 3)
        h, w = x_lr.shape[1], x_lr.shape[2]
        plt.imshow(model_generator.predict(x_lr[i].reshape(1, h, w, 3)).astype(np.uint8)[0])

    plt.savefig('data/srgan_generator_%s_%s.png' % (train_version, type_))
    plt.show()

In [ ]:
plt.figure(figsize=(15, 30))
for i in range(10):
    plt.subplot(10, 3, (i * 3) + 1)
    plt.imshow(x_test_lr[i])

    plt.subplot(10, 3, (i * 3) + 2)
    plt.imshow(x_test[i])

    plt.subplot(10, 3, (i * 3) + 3)
    w, h = x_test_lr.shape[1], x_test_lr.shape[2]
    plt.imshow(model_generator.predict(x_test_lr[i].reshape(1, w, h, 3)).astype(np.uint8)[0])

plt.savefig('data/srgan_generator_%s_test.png' % train_version)
plt.show()

## Discriminator network 

1. Input image
2. Conv2d with 3x3 kernel, 64 filters, and 1 stride
3. Leaky ReLU (https://keras.io/layers/advanced-activations/#leakyrelu)
3. Conv2d block
    1. Conv2d with 3x3 kernel, 64 filters, and 1 stride
    2. Batch normalization
    3. Leaky ReLU (https://keras.io/layers/advanced-activations/#leakyrelu)
4. Repeat 4 with 128 filters x 2 
5. Repeat 4 with 256 filters x 2 
6. Repeat 4 with 512 filters x 2 
7. Dense layer with 1024
8. Leaky ReLU
9. Dense 1 binary
10. Sigmoid

In [34]:
def srgan_discriminator():
    input_shape = 32, 32, 3
    features_1 = 64
    features_2, features_3, features_4 = 128, 256, 512
    kernel_size = 3, 3
    strides = 1, 1
    
    input_1 = Input(shape=input_shape)
    conv2d_2 = Conv2D(filters=features_1, kernel_size=kernel_size, strides=strides, padding='same')(input_1)
    lrelu_3 = LeakyReLU()(conv2d_2)
    
    conv2d_4_A = Conv2D(filters=features_1, kernel_size=kernel_size, strides=strides, padding='same')(lrelu_3)
    conv2d_4_B = BatchNormalization()(conv2d_4_A)
    lrelu_4_c = LeakyReLU()(conv2d_4_B)
    
    last_layer = lrelu_4_c
    for _features in [features_2, features_3, features_4]:
        for i in range(2):
            conv2d_5_A = Conv2D(filters=_features, kernel_size=kernel_size, strides=strides, 
                                padding='same')(last_layer)
            conv2d_5_B = BatchNormalization()(conv2d_5_A)
            lrelu_5_C = LeakyReLU()(conv2d_5_B)
            
            last_layer = lrelu_5_C
    
    flatten_8 = Flatten()(last_layer)
    dense_8 = Dense(1024)(flatten_8)
    lrelu_9 = LeakyReLU()(dense_8)
    dense_10 = Dense(1, activation='sigmoid')(lrelu_9)
    
    model_discriminator = Model(input_1, dense_10)
    model_discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model_discriminator

In [35]:
model_discriminator = srgan_discriminator()

In [44]:
generator_in = Input(shape=(16, 16, 3))

model_generator = srgan_generator((16, 16, 3), input_=generator_in)
model_discriminator = srgan_discriminator()

generator_out = model_generator(generator_in)

discriminator_out = model_discriminator(generator_out)
model_srgan = Model(generator_in, discriminator_out)

In [45]:
srgan_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_22 (InputLayer)        (None, 16, 16, 3)         0         
_________________________________________________________________
model_19 (Model)             (None, 32, 32, 3)         1736195   
_________________________________________________________________
model_20 (Model)             (None, 1)                 541565761 
Total params: 543,301,956
Trainable params: 543,294,020
Non-trainable params: 7,936
_________________________________________________________________


In [46]:
model_generator.compile(optimizer='adam', loss='mse')
model_discriminator.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_srgan.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [48]:
model_discriminator.layers